DATASETS

1. World Bank Open Data - Forest coverage (API)
2. UN Data - Population by sex and urban/rural residence between 2000 - 2020 (csv)
3. Food and Agriculture Organization of the United Nations (FAO) - FAOSTAT (csv)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re 
regex_pattern = re.compile(r'\s\((.*?)\)|(\s?,.*)')

Installed the wbdata library to access the World Bank Open Data API 

In [2]:
pip install wbdata

Note: you may need to restart the kernel to use updated packages.


Dataset #1 | Forest coverage

In [3]:
import wbdata
import datetime 

# Set the indicator code for forest coverage
indicator_code = 'AG.LND.FRST.ZS'

# Set the start and end year for the data
start_year = 2000
end_year = 2020

# Set the countries to retrieve data for
countries = ['USA', 'CAN', 'MEX']

# Retrieve the data
start_date = datetime.datetime(start_year, 1, 1)
end_date = datetime.datetime(end_year, 12, 31)

forest_data = wbdata.get_data(indicator_code, country=countries, data_date=(start_date, end_date))

# Convert the data to a pandas DataFrame
df = pd.DataFrame(forest_data)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator        63 non-null     object 
 1   country          63 non-null     object 
 2   countryiso3code  63 non-null     object 
 3   date             63 non-null     object 
 4   value            63 non-null     float64
 5   unit             63 non-null     object 
 6   obs_status       63 non-null     object 
 7   decimal          63 non-null     int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 4.1+ KB


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'CA', 'value': 'Canada'}",CAN,2020,38.695513,,,1
1,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'CA', 'value': 'Canada'}",CAN,2019,38.699637,,,1
2,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'CA', 'value': 'Canada'}",CAN,2018,38.703763,,,1
3,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'CA', 'value': 'Canada'}",CAN,2017,38.707888,,,1
4,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'CA', 'value': 'Canada'}",CAN,2016,38.712013,,,1


In [4]:
# Calculate the average forest coverage for each country
avg_forest_coverage = df.groupby('countryiso3code')['value'].mean()

# Print the average forest coverage for each country
print('Average forest coverage by country:')
print(avg_forest_coverage)

Average forest coverage by country:
countryiso3code
CAN    38.741693
MEX    34.463532
USA    33.631871
Name: value, dtype: float64


In [5]:
# Determine which country has the highest and lowest forest coverage
max_coverage = df.loc[df['value'].idxmax()]
min_coverage = df.loc[df['value'].idxmin()]

# Print the country with the highest and lowest forest coverage
print(f'The country with the highest forest coverage is {max_coverage["country"]} with a coverage of {max_coverage["value"]:.2f}%.')
print(f'The country with the lowest forest coverage is {min_coverage["country"]} with a coverage of {min_coverage["value"]:.2f}%.')

# Determine if any country has seen significant increases or decreases in forest coverage during the specified time period
for country in countries:
    country_data = df.loc[df['countryiso3code'] == country]
    initial_coverage = country_data.loc[country_data['date'] == str(start_year)]['value'].values[0]
    final_coverage = country_data.loc[country_data['date'] == str(end_year)]['value'].values[0]
    change = final_coverage - initial_coverage
    if abs(change) > 5:
        if change > 0:
            print(f'{country} has seen a significant increase in forest coverage of {change:.2f}% between {start_year} and {end_year}.')
        else:
            print(f'{country} has seen a significant decrease in forest coverage of {change:.2f}% between {start_year} and {end_year}.')

The country with the highest forest coverage is {'id': 'CA', 'value': 'Canada'} with a coverage of 38.79%.
The country with the lowest forest coverage is {'id': 'US', 'value': 'United States'} with a coverage of 33.13%.


In [6]:
# Retrieve the global forest coverage data for the specified time period
global_forest_data = wbdata.get_data(indicator_code,  data_date=(start_date, end_date))

global_forest_df = pd.DataFrame(global_forest_data)
global_forest_df.head()

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,30.174186,,,1
1,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,30.391558,,,1
2,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,30.611444,,,1
3,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,30.824248,,,1
4,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,31.039613,,,1


In [7]:
# Calculate the average global forest coverage 
global_avg_forest_coverage = df['value'].mean()
print(global_avg_forest_coverage)

# Compare the average forest coverage in the selected countries to the global average
if avg_forest_coverage.mean() > global_avg_forest_coverage:
    print('The selected countries have higher than average forest coverage compared to the global average.')
elif avg_forest_coverage.mean() < global_avg_forest_coverage:
    print('The selected countries have lower than average forest coverage compared to the global average.')
else:
    print('The selected countries have average forest coverage compared to the global average.')

35.61236560761129
The selected countries have higher than average forest coverage compared to the global average.


Country names

In [8]:
pip install pycountry pycountry_convert

Cleaning country names

In [9]:
def clean_country_name (name):
    if name is None:
        return name
    return re.sub(regex_pattern, '', name)

Assigning countries to continents

In [10]:
import pycountry
import pycountry_convert as pc

def country_to_continent (country_alpha2):        
    continent_code = pc.country_alpha2_to_continent_code (country_alpha2)    
    continent_name = pc.convert_continent_code_to_continent_name(continent_code)
    return continent_name

def country_name_to_continent (country_name):        
    try:
        country_code = pc.country_name_to_country_alpha2(country_name)    
        continent_name = country_to_continent(country_code)
        return continent_name
    except:        
        pass
    

# Create a lookup table mapping country names to continents
country_name_to_continent = {}
for country in pycountry.countries:  
    try:
        continent = country_to_continent(country.alpha_2)        
        country_name_to_continent[clean_country_name(country.name)]= continent         
    except:
        pass
print(country_name_to_continent)

{'Aruba': 'North America', 'Afghanistan': 'Asia', 'Angola': 'Africa', 'Anguilla': 'North America', 'Åland Islands': 'Europe', 'Albania': 'Europe', 'Andorra': 'Europe', 'United Arab Emirates': 'Asia', 'Argentina': 'South America', 'Armenia': 'Asia', 'American Samoa': 'Oceania', 'Antigua and Barbuda': 'North America', 'Australia': 'Oceania', 'Austria': 'Europe', 'Azerbaijan': 'Asia', 'Burundi': 'Africa', 'Belgium': 'Europe', 'Benin': 'Africa', 'Bonaire': 'North America', 'Burkina Faso': 'Africa', 'Bangladesh': 'Asia', 'Bulgaria': 'Europe', 'Bahrain': 'Asia', 'Bahamas': 'North America', 'Bosnia and Herzegovina': 'Europe', 'Saint Barthélemy': 'North America', 'Belarus': 'Europe', 'Belize': 'North America', 'Bermuda': 'North America', 'Bolivia': 'South America', 'Brazil': 'South America', 'Barbados': 'North America', 'Brunei Darussalam': 'Asia', 'Bhutan': 'Asia', 'Bouvet Island': 'Antarctica', 'Botswana': 'Africa', 'Central African Republic': 'Africa', 'Canada': 'North America', 'Cocos Isla

In [11]:
def country_code_to_name (row):
    code = row ['countryiso3code']
    try:
        return pycountry.countries.get(alpha_3=code).name
    except:
        pass

global_forest_df['country_name']=global_forest_df.apply(lambda row:country_code_to_name(row), axis=1)
global_forest_df.head()

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,country_name
0,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,30.174186,,,1,None
1,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,30.391558,,,1,None
2,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,30.611444,,,1,None
3,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,30.824248,,,1,None
4,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,31.039613,,,1,None


In [12]:
x=global_forest_df.filter(items=['country_name','date','value'])
x.head()
x['clean_country_name'] = x['country_name'].apply(lambda c:clean_country_name(c))

forest_valid_countries = x[x['clean_country_name'].isin(country_name_to_continent.keys())]


forest_valid_countries.head()
x.to_csv('forest.csv')

In [13]:
# add continent data to dataframe

forest_valid_countries['continent'] = forest_valid_countries['country_name'].map(country_name_to_continent)

avg_forest_coverage_by_continent = forest_valid_countries.groupby('continent')['value'].mean()
print(avg_forest_coverage_by_continent)
global_forest_df.info()
global_forest_df.head()

continent
Africa           27.834096
Asia             22.176845
Europe           30.732616
North America    37.781756
Oceania          47.350209
South America    51.207480
Name: value, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator        5586 non-null   object 
 1   country          5586 non-null   object 
 2   countryiso3code  5586 non-null   object 
 3   date             5586 non-null   object 
 4   value            5420 non-null   float64
 5   unit             5586 non-null   object 
 6   obs_status       5586 non-null   object 
 7   decimal          5586 non-null   int64  
 8   country_name     4515 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 392.9+ KB


<ipython-input-13-c3b6d5e37112>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forest_valid_countries['continent'] = forest_valid_countries['country_name'].map(country_name_to_continent)


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,country_name
0,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,30.174186,,,1,None
1,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,30.391558,,,1,None
2,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,30.611444,,,1,None
3,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,30.824248,,,1,None
4,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,31.039613,,,1,None


In [14]:
global_forest_df = global_forest_df.groupby(['date', 'country_name']).sum('value')
global_forest_df.reset_index(inplace=True)
global_forest_df['Year'] = global_forest_df['date'].astype(int)
global_forest_df=global_forest_df.rename(columns={'value':'forest coverage'})
global_forest_df.info()
global_forest_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4515 entries, 0 to 4514
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             4515 non-null   object 
 1   country_name     4515 non-null   object 
 2   forest coverage  4515 non-null   float64
 3   decimal          4515 non-null   int64  
 4   Year             4515 non-null   int32  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 158.9+ KB


,date,country_name,forest coverage,decimal,Year
0,2000,Afghanistan,1.852782,1,2000
1,2000,Albania,28.076642,1,2000
2,2000,Algeria,0.662961,1,2000
3,2000,American Samoa,88.650000,1,2000
4,2000,Andorra,34.042553,1,2000


Dataset #2 | Population

In [15]:
population = pd.read_csv("population_by_sex_residence.csv", index_col = 0)
population.head()

,Year,Area,Sex,Record Type,Reliability,Source Year,Value,Value Footnotes
Country or Area,,,,,,,,
Afghanistan,2020,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2021.0,31390171.0,1
Afghanistan,2019,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2019.0,30725560.0,1
Afghanistan,2018,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2019.0,30075018.0,1
Afghanistan,2017,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2018.0,28224323.0,1
Afghanistan,2016,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2017.0,27657145.0,1


In [16]:
population.columns
population.reset_index(inplace=True)
population.columns
population.head()

,Country or Area,Year,Area,Sex,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Afghanistan,2020,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2021.0,31390171.0,1
1,Afghanistan,2019,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2019.0,30725560.0,1
2,Afghanistan,2018,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2019.0,30075018.0,1
3,Afghanistan,2017,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2018.0,28224323.0,1
4,Afghanistan,2016,Total,Both Sexes,Estimate - de facto,"Final figure, incomplete/questionable reliability",2017.0,27657145.0,1


In [17]:
population = population.filter(items=['Country or Area','Year','Reliability','Value']).loc[(population['Reliability']=='Final figure, complete')]
population['Year'] = population['Year'].astype(int)
population = population.loc[(population['Year']>=2000) & (population['Year']<2020)]
population=population.rename(columns={'Value':'population value'})
population.head()

,Country or Area,Year,Reliability,population value
22,Åland Islands,2019,"Final figure, complete",29836.5
23,Åland Islands,2018,"Final figure, complete",29638.0
24,Åland Islands,2017,"Final figure, complete",29352.0
25,Åland Islands,2016,"Final figure, complete",29099.0
26,Åland Islands,2015,"Final figure, complete",28950.0


In [18]:
population = population.groupby(['Year', 'Country or Area']).mean('population value')


population.reset_index(inplace=True)
population.columns
population.head()

,Year,Country or Area,population value
0,2000,Albania,3.060908e+06
1,2000,American Samoa,5.749550e+04
2,2000,Andorra,6.586533e+04
3,2000,Anguilla,1.125400e+04
4,2000,Antigua and Barbuda,7.231000e+04


In [19]:
population.to_csv("populationWithContinent.csv")

First merged DataFrame | Population & Forest coverage on 'valid' countries 

In [20]:
merged_df_1 = pd.merge(population, global_forest_df, left_on=['Country or Area','Year'], right_on=['country_name', 'Year'], how='inner')
print(merged_df_1.shape)
merged_df_1.head()

(2112, 7)


,Year,Country or Area,population value,date,country_name,forest coverage,decimal
0,2000,Albania,3.060908e+06,2000,Albania,28.076642,1
1,2000,American Samoa,5.749550e+04,2000,American Samoa,88.650000,1
2,2000,Andorra,6.586533e+04,2000,Andorra,34.042553,1
3,2000,Antigua and Barbuda,7.231000e+04,2000,Antigua and Barbuda,21.477273,1
4,2000,Argentina,3.678386e+07,2000,Argentina,12.196486,1


In [21]:
merged_df_1.to_csv("merged_df_1.csv")

In [22]:
counts = merged_df_1['country_name'].value_counts()
valid_countries = counts[counts >= 20].index.tolist()

merged_df_1_filtered = merged_df_1[merged_df_1['country_name'].isin(valid_countries)]


merged_df_1_filtered.head()

,Year,Country or Area,population value,date,country_name,forest coverage,decimal
0,2000,Albania,3.060908e+06,2000,Albania,28.076642,1
1,2000,American Samoa,5.749550e+04,2000,American Samoa,88.650000,1
2,2000,Andorra,6.586533e+04,2000,Andorra,34.042553,1
3,2000,Antigua and Barbuda,7.231000e+04,2000,Antigua and Barbuda,21.477273,1
4,2000,Argentina,3.678386e+07,2000,Argentina,12.196486,1


In [29]:
merged_df_1_filtered.to_csv("merged_filtered.csv")

Dataset #3

In [24]:
food = pd.read_csv("food_production.csv", index_col = 0)
food.head()

C:\Users\bogla_d7nsnlv\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
Area Code,,,,,,,,,,,,
2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1975,1975,ha,0.0,E
2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1976,1976,ha,5900.0,E
2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1977,1977,ha,6000.0,E
2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1978,1978,ha,6000.0,E
2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1979,1979,ha,6000.0,E


In [25]:
filtered_food= food.filter(items=['Area','Item','Element','Year','Unit','Value','Flag']).loc[(food['Flag']=='A')& (food['Element']=='Area harvested') & (food['Year']>=2000) & (food['Year']<2020)]
food_summary=filtered_food.groupby(['Year', 'Area']).sum('Value')
food_summary.reset_index(inplace=True)
food_summary.head()


food_valid_countries = food_summary[food_summary['Area'].isin(country_name_to_continent.keys())]
print(food_valid_countries)

      Year         Area       Value
0     2000  Afghanistan   4859700.0
2     2000      Albania    461394.0
3     2000      Algeria   2845476.0
5     2000       Angola   2530960.0
6     2000    Argentina  35446241.0
...    ...          ...         ...
3826  2019   Uzbekistan   3359680.0
3828  2019     Viet Nam  24316319.0
3833  2019        Yemen   1613752.0
3834  2019       Zambia   2723038.0
3835  2019     Zimbabwe   1796163.0

[2905 rows x 3 columns]


In [26]:
country_counts_food = food_valid_countries['Area'].value_counts()
filtered_food_valid_countries = food_valid_countries[food_valid_countries['Area'].isin(country_counts_food[country_counts_food == 20].index)]


print(filtered_food_valid_countries)

      Year         Area       Value
0     2000  Afghanistan   4859700.0
2     2000      Albania    461394.0
3     2000      Algeria   2845476.0
5     2000       Angola   2530960.0
6     2000    Argentina  35446241.0
...    ...          ...         ...
3825  2019      Uruguay   2444806.0
3826  2019   Uzbekistan   3359680.0
3828  2019     Viet Nam  24316319.0
3833  2019        Yemen   1613752.0
3834  2019       Zambia   2723038.0

[2300 rows x 3 columns]


In [27]:
filtered_food_valid_countries.groupby(['Area']).count()
filtered_food_valid_countries=filtered_food_valid_countries.rename(columns={'Value':'Area harvested - ha'})
filtered_food_valid_countries.head()

,Year,Area,Area harvested - ha
0,2000,Afghanistan,4859700.0
2,2000,Albania,461394.0
3,2000,Algeria,2845476.0
5,2000,Angola,2530960.0
6,2000,Argentina,35446241.0


In [28]:
merged_df_2 = merged_df_2.groupby(['Year', 'Country or Area']).mean('population value')
merged_df_2.reset_index(inplace=True)
merged_df_2.head()

NameError: name 'merged_df_2' is not defined

In [ ]:
final_df= merged_df_2.filter(items=['Year','Country or Area','continent','population value','forest coverage','Area harvested - ha'])

print(final_df.shape)
final_df.head()

In [ ]:
final_df=final_df[final_df['Country or Area'].isin(final_df[final_df == 20].index)]
print(final_df.shape)
final_df.head()

In [ ]:
final_df.to_csv('final_df.csv')